In [1]:
using LightGraphs
using MatrixNetworks
using VegaDatasets
using DataFrames
using SparseArrays
using LinearAlgebra
using Plots
using VegaLite

LoadError: ArgumentError: Package LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [2]:
airports = dataset("airports")
flightsairport = dataset("flights-airport")

LoadError: UndefVarError: dataset not defined

In [3]:
airports

LoadError: UndefVarError: airports not defined

In [4]:
flightsairportdf = DataFrame(flightsairport)

LoadError: UndefVarError: DataFrame not defined

In [5]:
allairports = vcat(flightsairportdf[!,:origin],flightsairportdf[!,:destination])
uairports = unique(allairports)

LoadError: UndefVarError: flightsairportdf not defined

In [6]:
# create an airports data frame that has a subset of airports that are only included in the routes dataset
airportsdf = DataFrame(airports)
subsetairports = map(i->findfirst(airportsdf[!, :iata].==uairports[i]),1:length(uairports))
airportsdf_subset = airportsdf[subsetairports,:]

LoadError: UndefVarError: DataFrame not defined

In [7]:
# build the adjacency matrix
ei_ids = findfirst.(isequal.(flightsairportdf[!,:origin]), [uairports])
ej_ids = findfirst.(isequal.(flightsairportdf[!,:destination]), [uairports])
edgeweights = flightsairportdf[!,:count]
;

LoadError: UndefVarError: flightsairportdf not defined

In [8]:
A = sparse(ei_ids,ej_ids,1,length(uairports),length(uairports))
A = max.(A,A')

LoadError: UndefVarError: uairports not defined

In [9]:
spy(A)

LoadError: UndefVarError: spy not defined

In [10]:
issymmetric(A)

LoadError: UndefVarError: issymmetric not defined

There is one popular graphs package in julia and that is the `LightGraphs` package. It does offer a lot of functionality and encourage you to check it out. Nevertheless, the operations we will need here will be basic graph analysis operations and we will make use of the `MatrixNetworks` package. For future reference, and for when you have a "graphs-heavy" project, here's a quick example of how you would build a graph via `LightGraphs`.

In [11]:
L = SimpleGraph(A)

LoadError: UndefVarError: SimpleGraph not defined

In [12]:
G=SimpleGraph(10) #SimpleGraph(nnodes,nedges) 
add_edge!(G,7,5)#modifies graph in place.
add_edge!(G,3,5)
add_edge!(G,5,2)

LoadError: UndefVarError: SimpleGraph not defined

For now, we will go back to use the graph's adjacency matrix to perform all the operations we want to perform.

The first is finding the connected components of the graph. Often, it is important to know if the graph you are given is connected specially if you want to later perform diffusion operations on it. Think of it this way: if the graph is disconnected, this means that there is no way for information to travel from one component to another.

#### scomponents

In [13]:
cc = scomponents(A)

LoadError: UndefVarError: scomponents not defined

cc.sizes will show the size of each component found. Here, only one component was found and it contains all 305 nodes. This is great, because this means that the graph is connected. 

Next, we will look at the degree distribution of this graph.

In [14]:
degrees = sum(A,dims=2)[:]
p1 = plot(sort(degrees,rev=true),ylabel="log degree",legend=false,yaxis=:log)
p2 = plot(sort(degrees,rev=true),ylabel="degree",legend=false)
plot(p1,p2,size=(600,300))

LoadError: UndefVarError: A not defined

This is actually very interesting because it looks like that the airline transportation network seems to fit a powerlaw degree distribution. Knowing that your graph fits a well known model for degree distribution can be very helpful for further studying it. (For instance, there is a lot of literature on graphs that fit power law degree distributions).

Out of curiosity, let's find the airport that has the most connections

In [15]:
maxdegreeid = argmax(degrees)
uairports[maxdegreeid]

LoadError: UndefVarError: degrees not defined

In [16]:
us10m = dataset("us-10m")
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:lightgray,
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=10},
    color={value=:steelblue}
)+
@vlplot(
    :rule,
    data=flightsairport,
    transform=[
        {filter={field=:origin,equal=:ATL}},
        {
            lookup=:origin,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["origin_latitude", "origin_longitude"]
        },
        {
            lookup=:destination,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["dest_latitude", "dest_longitude"]
        }
    ],
    projection={type=:albersUsa},
    longitude="origin_longitude:q",
    latitude="origin_latitude:q",
    longitude2="dest_longitude:q",
    latitude2="dest_latitude:q"
)

LoadError: UndefVarError: dataset not defined

### 🟡Shortest path problem
Finding the shortest path between two nodes. We will use Dijkstra's algorithm.

In [17]:
ATL_paths = dijkstra(A,maxdegreeid)

LoadError: UndefVarError: dijkstra not defined

We will find a node that has the longest distance from ATL.

In [18]:
ATL_paths[1][maxdegreeid]

LoadError: UndefVarError: ATL_paths not defined

In [19]:
maximum(ATL_paths[1])

LoadError: UndefVarError: ATL_paths not defined

In [20]:
@show stop1 = argmax(ATL_paths[1])
@show uairports[stop1]
;

LoadError: UndefVarError: ATL_paths not defined

In [21]:
@show stop2 = ATL_paths[2][stop1]
@show uairports[stop2]
;

LoadError: UndefVarError: ATL_paths not defined

In [22]:
@show stop3 = ATL_paths[2][stop2]
@show uairports[stop3]
;

LoadError: UndefVarError: ATL_paths not defined

In [23]:
@show stop4 = ATL_paths[2][stop3]
@show uairports[stop4]
;

LoadError: UndefVarError: ATL_paths not defined

In [24]:
using VegaLite, VegaDatasets

us10m = dataset("us-10m")
airports = dataset("airports")

@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=5},
    color={value=:gray}
) +
@vlplot(
    :line,
    data={
        values=[
            {airport=:ATL,order=1},
            {airport=:SEA,order=2},
            {airport=:JNU,order=3},
            {airport=:GST,order=4}
        ]
    },
    transform=[{
        lookup=:airport,
        from={
            data=airports,
            key=:iata,
            fields=["latitude","longitude"]
        }
    }],
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    order={field=:order,type=:ordinal}
)

LoadError: ArgumentError: Package VegaLite [112f6efa-9a02-5b7d-90c0-432ed331239a] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


We can look at distances from other airports too, and put the whole thing in a function.

In [25]:
nodeid = argmin(degrees)
@show uairports[nodeid]
d = dijkstra(A,nodeid)
argmax(d[1]),uairports[argmax(d[1])]

LoadError: UndefVarError: degrees not defined

In [26]:
function find_path(d,id)
    shortestpath = zeros(Int,1+Int.(d[1][id]))
    shortestpath[1] = id
    for i = 2:length(shortestpath)
        shortestpath[i] = d[2][shortestpath[i-1]]
    end
    return shortestpath
end
p = find_path(d,123)
uairports[p]

LoadError: UndefVarError: d not defined

### 🟡Minimum Spanning Tree (MST)
The next problem is forming a minimum spanning tree on the graph. The idea of a minimum spanning tree is to connect all nodes in the graph with as little edges as possible. We will use Prim's algorithm for this problem.

In [27]:
?mst_prim

search:

Couldn't find mst_prim
Perhaps you meant lstrip


No documentation found.

Binding `mst_prim` does not exist.


In [28]:
ti,tj,tv,nverts = mst_prim(A)

LoadError: UndefVarError: mst_prim not defined

We will form a new DataFrame with a new set of edges (from the MST)

In [29]:
df_edges = DataFrame(:ei=>uairports[ti],:ej=>uairports[tj])

LoadError: UndefVarError: uairports not defined

In [30]:
@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=20},
    color={value=:gray}
) +
@vlplot(
    :rule,
    data=df_edges, #data=flightsairport,
    transform=[
        {
            lookup=:ei,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["originx", "originy"]
        },
        {
            lookup=:ej,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["destx", "desty"]
        }
    ],
    projection={type=:albersUsa},
    longitude="originy:q",
    latitude="originx:q",
    longitude2="desty:q",
    latitude2="destx:q"
)

LoadError: LoadError: UndefVarError: @vlplot not defined
in expression starting at In[30]:1

### 🟡PageRank
PageRank is the algorithm that got Google started. The idea is: given an network of connections between multiple nodes (web pages in the cae of Google), is there a way to return a list of ranked nodes? PageRank provides this ranking. Obviously, nodes can be ranked in several different ways but PageRank remains to be one of the most popular methods in network analysis. Let's check out the documentation of `pagerank` below.

In [31]:
?MatrixNetworks.pagerank

UndefVarError: UndefVarError: MatrixNetworks not defined

Now, we will run PageRank on our adjacency matrix `A`.

In [32]:
v = MatrixNetworks.pagerank(A,0.85)

LoadError: UndefVarError: MatrixNetworks not defined

Here, note that the return vector `v` sums to `1`.

In [33]:
sum(v)

LoadError: UndefVarError: v not defined

In [34]:
insertcols!(airportsdf_subset,7,:pagerank_value=>v)

LoadError: UndefVarError: v not defined

In [35]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size="pagerank_value:q",
    color={value=:steelblue}
)

LoadError: LoadError: UndefVarError: @vlplot not defined
in expression starting at In[35]:1

### 🟡Clustering Coefficients
From Wikipedia: The local clustering coefficient of a vertex (node) in a graph quantifies how close its neighbours are to being a clique (complete graph).

This means that if for example, a node is connected to two nodes that are also connected to each other, that node's clustering coeefficient is 1. This can be a good metric to find out which nodes tend to have tight clusters around them. Let's look at the documentation of `clustercoeffs` from MatrixNetworks.

In [36]:
?clustercoeffs

search:

Couldn't find clustercoeffs
Perhaps you meant countlines


No documentation found.

Binding `clustercoeffs` does not exist.


In [37]:
cc = clustercoeffs(A)
cc[findall(cc.<=eps())] .= 0
cc

LoadError: UndefVarError: clustercoeffs not defined

In [38]:
insertcols!(airportsdf_subset,7,:ccvalues=>cc)

LoadError: UndefVarError: cc not defined

In [39]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size="ccvalues:q",
    color={value=:gray}
)

LoadError: LoadError: UndefVarError: @vlplot not defined
in expression starting at In[39]:1

# Finally...
After finishing this notebook, you should be able to:
- [ ] take a list of connections between nodes and form an adjacency matrix out of them
- [ ] use the LightGraphs package to create a graph
- [ ] detect if a graph is connected or not
- [ ] solve the shortest path problem on a graph and a given node
- [ ] solve the minimum spanning tree problem on a graph
- [ ] solve the PageRank problem on a graph
- [ ] find the clustering coefficients of nodes in a graph